# Context Managers

In [1]:
with open('data.txt', 'w') as fobj:
    print(fobj.closed)
print(fobj.closed)

False
True


In [2]:
class MyContext:
    
    def __enter__(self):
        print('before')
        return 42
    
    # signature: exception type and value, and traceback
    def __exit__(self, ex_t, ex_v, tb):
        print('after')
        print(ex_t, ex_v, tb)

In [3]:
with MyContext() as context_obj:
    print('doing something with ', context_obj)

before
doing something with  42
after
None None None


In [4]:
with MyContext() as context_obj:
    print('doing something with ', context_obj)
    1/0
    print('after exception')

before
doing something with  42
after
<class 'ZeroDivisionError'> division by zero <traceback object at 0x10fba5e48>


ZeroDivisionError: division by zero

In [5]:
class MyContext2:
    
    def __enter__(self):
        print('before')
        return 42
    
    # signature: exception type and value, and traceback
    def __exit__(self, ex_t, ex_v, tb):
        print('after')
        print(ex_t, ex_v, tb)
        return True

In [7]:
with MyContext2() as context_obj:
    print('doing something with ', context_obj)
    1/0
    print('after exception')

before
doing something with  42
after
<class 'ZeroDivisionError'> division by zero <traceback object at 0x10fb81f48>


## Contextlib

In [8]:
import contextlib

In [9]:
@contextlib.contextmanager
def mycontext():
    print('before')
    try:
        yield 42
    finally:
        print('after')

In [10]:
with mycontext() as ctx:
    print('doing things')

before
doing things
after


### examples

In [11]:
class Test:
    
    def close(self):
        print('closing')

In [12]:
with contextlib.closing(Test()) as ctx:
    print('working with', ctx)

working with <__main__.Test object at 0x10fc2e1d0>
closing


In [13]:
# More interesting methods
contextlib.ExitStack?

Init signature: contextlib.ExitStack()
Docstring:     
Context manager for dynamic management of a stack of exit callbacks.

For example:
    with ExitStack() as stack:
        files = [stack.enter_context(open(fname)) for fname in filenames]
        # All opened files will automatically be closed at the end of
        # the with statement, even if attempts to open files later
        # in the list raise an exception.
File:           ~/miniconda3/envs/py37env/lib/python3.7/contextlib.py
Type:           ABCMeta


In [22]:
with open('total.txt', 'w') as fobj_out:
    with open('file1.txt') as fobj_in_1:
        with open('file2.txt') as fobj_in_2:
            fobj_out.write(fobj_in_1.read())
            fobj_out.write(fobj_in_2.read())

In [23]:
%less total.txt

file1
file2


In [25]:
with open('total.txt', 'w') as fobj_out, open('file1.txt') as fobj_in_1, open('file2.txt') as fobj_in_2:
    fobj_out.write(fobj_in_1.read())
    fobj_out.write(fobj_in_2.read())

In [26]:
%less total.txt

file1
file2


In [27]:
with (open('total.txt', 'w') as fobj_out,
      open('file1.txt') as fobj_in_1,
      open('file2.txt') as fobj_in_2):
    fobj_out.write(fobj_in_1.read())
    fobj_out.write(fobj_in_2.read())

SyntaxError: invalid syntax (<ipython-input-27-1ec30ad9ea42>, line 1)

### Exercice 5.5.1

In [41]:
import sys

class MyContext:
    
    def __enter__(self):
        self.fobj = open('stdout.txt', 'w')
        self._stdout = sys.stdout
        sys.stdout = self.fobj
    
    # signature: exception type and value, and traceback
    def __exit__(self, ex_t, ex_v, tb):
        sys.stdout = self._stdout
        self.fobj.close()
        
with MyContext() as ctx:
    print('inside the file')
print('in command line')

In [42]:
%save ex551.py 41

File `ex551.py` exists. Overwrite (y/[N])?   y


### Exercice 5.5.2

In [70]:
import sys
import contextlib

@contextlib.contextmanager
def mycontext():
    
    try:
        fobj = open('stdout.txt', 'w')
        stdout = sys.stdout
        sys.stdout = fobj
        yield
    
    finally:
        sys.stdout = stdout
        fobj.close()
        
with mycontext() as ctx:
    print('inside the file')
print('in command line')

In [71]:
%save ex552.py 70

File `ex552.py` exists. Overwrite (y/[N])?   y


### Exercice 5.5.3

In [72]:
%ls

In [1]:
import sys
import os

class MyContext():
    def __init__(self, path):
        self.path = path
    def __enter__(self):
        #os.makedirs(self.path)
        self._curdir = os.path.abspath(os.curdir)
        os.chdir(os.path.join(self._curdir, self.path))
    # signature: exception type and value, and traceback
    def __exit__(self, ex_t, ex_v, tb):
        os.chdir(self._curdir)

print("Current path is: ",os.path.abspath(os.curdir))
with MyContext('test') as ctx:
    print("Inside contextmanager path is: ",os.path.abspath(os.curdir))
print("Current path is: ",os.path.abspath(os.curdir))

Current path is:  /Volumes/HDD-Data/PycharmProjects/AdvancedPython/05-Context-Managers
Inside contextmanager path is:  /Volumes/HDD-Data/PycharmProjects/AdvancedPython/05-Context-Managers/test
Current path is:  /Volumes/HDD-Data/PycharmProjects/AdvancedPython/05-Context-Managers


### Exercice 5.5.4

In [8]:
import sys
import os
import contextlib

@contextlib.contextmanager
def mycontext(path):
    _path = path
    try:
        _curdir = os.path.abspath(os.curdir)
        os.chdir(os.path.join(_curdir, _path))
        yield
    finally:
        os.chdir(_curdir)

print("Current path is: ",os.path.abspath(os.curdir))
with mycontext('test') as ctx:
    print("Inside contextmanager path is: ",os.path.abspath(os.curdir))
print("Current path is: ",os.path.abspath(os.curdir))

Current path is:  /Volumes/HDD-Data/PycharmProjects/AdvancedPython/05-Context-Managers
Inside contextmanager path is:  /Volumes/HDD-Data/PycharmProjects/AdvancedPython/05-Context-Managers/test
Current path is:  /Volumes/HDD-Data/PycharmProjects/AdvancedPython/05-Context-Managers


### Exercice 5.5.5